In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import io
import os
import math

In [2]:
! ls $PWD

'Preprocessing Listings.csv2.ipynb'  'Preprocessing Listings.csv.ipynb'
'Preprocessing Listings.csv3.ipynb'


In [9]:
listingsOr = pd.read_csv('../Data/listings.csv')

# We start studying and cleaning listings df

In [10]:
listingsOr.describe()

,id,scrape_id,thumbnail_url,medium_url,xl_picture_url,host_id,host_acceptance_rate,host_listings_count,host_total_listings_count,latitude,...,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,jurisdiction_names,calculated_host_listings_count,reviews_per_month
count,1.631300e+04,1.631300e+04,0.0,0.0,0.0,1.631300e+04,0.0,16284.000000,16284.000000,16313.000000,...,13118.000000,13105.000000,13111.000000,13089.000000,13104.000000,13083.000000,13082.000000,0.0,16313.000000,13261.000000
mean,1.406692e+07,2.018012e+13,NaN,NaN,NaN,5.516152e+07,NaN,13.053058,13.053058,40.420583,...,92.426666,9.484624,9.378003,9.667431,9.658501,9.578766,9.248433,NaN,8.575921,2.062498
std,6.700650e+06,4.355602e+00,NaN,NaN,NaN,4.934384e+07,NaN,48.033478,48.033478,0.020241,...,8.857210,0.892461,0.975935,0.765794,0.797749,0.751881,0.945927,NaN,22.955640,2.641989
min,1.862800e+04,2.018012e+13,NaN,NaN,NaN,5.154000e+03,NaN,0.000000,0.000000,40.321048,...,20.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,NaN,1.000000,0.020000
25%,8.791775e+06,2.018012e+13,NaN,NaN,NaN,1.096953e+07,NaN,1.000000,1.000000,40.409933,...,90.000000,9.000000,9.000000,10.000000,10.000000,9.000000,9.000000,NaN,1.000000,0.510000
50%,1.584329e+07,2.018012e+13,NaN,NaN,NaN,3.890332e+07,NaN,2.000000,2.000000,40.418763,...,95.000000,10.000000,10.000000,10.000000,10.000000,10.000000,9.000000,NaN,2.000000,1.360000
75%,1.955322e+07,2.018012e+13,NaN,NaN,NaN,9.618825e+07,NaN,6.000000,6.000000,40.427577,...,98.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,NaN,5.000000,2.990000
max,2.277218e+07,2.018012e+13,NaN,NaN,NaN,1.681233e+08,NaN,733.000000,733.000000,40.604738,...,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,NaN,179.000000,155.740000


In [11]:
#Potential columns to drop:
columns_to_drop = ['listing_url','scrape_id', 'last_scraped', 'scrape_id', 'last_scraped', 'name', 
                   'experiences_offered', 'picture_url','host_url', 'host_name','host_thumbnail_url', 
                   'host_picture_url', 'host_location','summary', 'space', 'description', 'neighbourhood', 
                   'neighbourhood_group_cleansed','city','state', 'zipcode', 'market', 'smart_location',
                   'neighborhood_overview','notes', 'transit', 'access', 'interaction', 'house_rules', 
                   'host_id', 'host_about', 'license', 'requires_license', 'host_neighbourhood','host_verifications',
                   'street', 'country_code', 'is_location_exact', 'guests_included', 'calendar_updated', 
                   'has_availability','calendar_last_scraped', 'host_total_listings_count', 'maximum_nights','country']
#remove for Nan
rows_to_drop=['bedrooms','bathrooms','beds','review_scores_value', 'latitude', 'longitude', 'neighbourhood_cleansed']

In [12]:
listings = listingsOr.copy()

In [13]:
#We delete all columns that are all NaN
listings=listings.dropna(axis=1,how='all')

In [14]:
#for column in columns_to_drop:
listings=listings.drop(columns_to_drop,axis=1)

In [15]:
listings=listings.dropna(axis=0,how='any',subset=rows_to_drop)

In [16]:
listings.shape

(13048, 48)

### We make sure there are no null values in the features we are going to use

In [17]:
listings.isnull().sum()

id                                      0
host_since                             18
host_response_time                   1242
host_response_rate                   1242
host_is_superhost                      18
host_listings_count                    18
host_has_profile_pic                   18
host_identity_verified                 18
neighbourhood_cleansed                  0
latitude                                0
longitude                               0
property_type                           0
room_type                               0
accommodates                            0
bathrooms                               0
bedrooms                                0
beds                                    0
bed_type                                0
amenities                               0
square_feet                         12636
price                                   0
weekly_price                        10477
monthly_price                       10628
security_deposit                  

In [18]:
#We supose null values are no superhost
listings['host_is_superhost'].fillna('f', inplace=True)
#listings['host_is_superhost']=np.where(listings['host_is_superhost']=='f', 0, 1)

In [19]:
#We supose null values are no picture
listings['host_has_profile_pic'].fillna('f', inplace=True)
#listings['host_has_profile_pic']=np.where(listings['host_has_profile_pic']=='f', 0, 1)

In [20]:
#We supose null values are no identity verified
listings['host_identity_verified'].fillna('f', inplace=True)
#listings['host_identity_verified']=np.where(listings['host_identity_verified']=='f', 0, 1)

In [21]:
#We would remove square_feet column as there are too many missing values, but we are not going to use this column
#listings=listings.drop('square_feet', axis=1)

In [22]:
#We convert weekly price and monthly price to boolean, as there are too many missing values,
#and being missing might mean that they do not have a different price
listings['weekly_price']=(listings['weekly_price'].str.replace(r'[^\d.]','').astype(float))
listings['weekly_price'].fillna(0, inplace=True)
listings['weekly_price']=np.where(listings['weekly_price']>0, 't', 'f')

In [23]:
listings['monthly_price']=(listings['monthly_price'].str.replace(r'[^\d.]','').astype(float))
listings['monthly_price'].fillna(0, inplace=True)
listings['monthly_price']=np.where(listings['monthly_price']>0, 't', 'f')

In [24]:
#We supose having missing fee might mean there is no fee
listings['security_deposit']=(listings['security_deposit'].str.replace(r'[^\d.]','').astype(float))
listings['security_deposit'].fillna(0, inplace=True)
listings['cleaning_fee']=(listings['cleaning_fee'].str.replace(r'[^\d.]','').astype(float))
listings['cleaning_fee'].fillna(0, inplace=True)

In [25]:
#We remove all rows where any of the "scores" columns is missing
reviewScoresColums = listings.filter(regex='review_scores.*').columns
listings=listings.dropna(axis=0,how='any', subset=reviewScoresColums)

In [26]:
#We supose if there is no information about the number of houses rented by the host, it is at least for sure 1
listings['host_listings_count'].fillna(1, inplace=True)

In [27]:
#We fill the missing values with the mean and the mode
listings['host_response_rate']=(listings['host_response_rate'].str.replace("%",'').astype(float))/100
listings['host_response_rate'].fillna(listings['host_response_rate'].median(),inplace=True)
listings['host_response_time'].fillna('within an hour',inplace=True)

In [28]:
#Only missing values in host_since and square_feet columns
listings.isnull().sum()

id                                      0
host_since                             18
host_response_time                      0
host_response_rate                      0
host_is_superhost                       0
host_listings_count                     0
host_has_profile_pic                    0
host_identity_verified                  0
neighbourhood_cleansed                  0
latitude                                0
longitude                               0
property_type                           0
room_type                               0
accommodates                            0
bathrooms                               0
bedrooms                                0
beds                                    0
bed_type                                0
amenities                               0
square_feet                         12631
price                                   0
weekly_price                            0
monthly_price                           0
security_deposit                  

In [29]:
listings = listings.reset_index()

### We check the data types are right

In [30]:
listings.dtypes

index                                 int64
id                                    int64
host_since                           object
host_response_time                   object
host_response_rate                  float64
host_is_superhost                    object
host_listings_count                 float64
host_has_profile_pic                 object
host_identity_verified               object
neighbourhood_cleansed               object
latitude                            float64
longitude                           float64
property_type                        object
room_type                            object
accommodates                          int64
bathrooms                           float64
bedrooms                            float64
beds                                float64
bed_type                             object
amenities                            object
square_feet                         float64
price                                object
weekly_price                    

#### Numeric variables

In [31]:
numeric_features = ['id','latitude', 'longitude','host_response_rate','host_listings_count', 'accommodates', 'bathrooms',
       'bedrooms', 'beds', 'price', 'security_deposit', 'cleaning_fee', 'extra_people',
       'minimum_nights', 'availability_30', 'availability_60',
       'availability_90', 'availability_365', 'number_of_reviews','review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value','calculated_host_listings_count', 'reviews_per_month']

In [32]:
listings['price']=(listings['price'].str.replace(r'[^\d.]','').astype(float))

In [33]:
listings['extra_people']=(listings['extra_people'].str.replace(r'[^\d.]','').astype(float))

In [34]:
listings.shape

(13042, 49)

#### Amenities

In [35]:
listings['amenities'] = listings['amenities'].map(
    lambda amns: "|".join([amn.replace("}", "").replace("{", "").replace('"', "")\
                           for amn in amns.split(",")])
)

In [36]:
amenities = np.unique(np.concatenate(listings['amenities'].map(lambda amns: amns.split("|")).values))[1:-2] 
#First amenity is empty string and last 2 say "translation missing"
amenities_matrix = np.array([listings['amenities'].map(lambda amns: 1 if amn in amns else 0) for amn in amenities])

In [37]:
Amenities_columns =np.array(["Amen_" + amenity  for amenity in amenities])

In [38]:
Amenities_columns

array(['Amen_ toilet', 'Amen_24-hour check-in',
       'Amen_Accessible-height bed', 'Amen_Accessible-height toilet',
       'Amen_Air conditioning', 'Amen_BBQ grill', 'Amen_Baby bath',
       'Amen_Baby monitor', 'Amen_Babysitter recommendations',
       'Amen_Bathtub', 'Amen_Bathtub with shower chair',
       'Amen_Beach essentials', 'Amen_Bed linens', 'Amen_Breakfast',
       'Amen_Buzzer/wireless intercom', 'Amen_Cable TV',
       'Amen_Carbon monoxide detector', 'Amen_Cat(s)',
       'Amen_Changing table', 'Amen_Children’s books and toys',
       'Amen_Children’s dinnerware', 'Amen_Cleaning before checkout',
       'Amen_Coffee maker', 'Amen_Cooking basics', 'Amen_Crib',
       'Amen_Disabled parking spot', 'Amen_Dishes and silverware',
       'Amen_Dishwasher', 'Amen_Dog(s)', 'Amen_Doorman', 'Amen_Dryer',
       'Amen_EV charger', 'Amen_Elevator', 'Amen_Essentials',
       'Amen_Ethernet connection', 'Amen_Extra pillows and blankets',
       'Amen_Family/kid friendly', 'Amen_Fire

In [39]:
Amenities_DF = pd.DataFrame(data=amenities_matrix.T, columns=Amenities_columns)
#As there are too many different amenities, we keep only the 2 ones that we consider could make a difference (this
#coud be investigated deeper)
Amenities_DF['Wifi'] = Amenities_DF[['Amen_Wireless Internet','Amen_Internet','Amen_Pocket wifi']].max(axis=1)
Amenities_DF['Parking']=Amenities_DF['Amen_Paid parking off premises']

#### Categorical variables

In [40]:
categorical_features = ['neighbourhood_cleansed', 'property_type', 'bed_type','room_type','cancellation_policy']

In [41]:
features = pd.concat([listings[numeric_features],listings[categorical_features],Amenities_DF['Wifi'],Amenities_DF['Parking']], axis=1)

#### Boolean categories

In [42]:
for tf_feature in ['host_is_superhost', 'host_identity_verified', 'host_has_profile_pic',
                   'instant_bookable','require_guest_profile_picture', 'require_guest_phone_verification', 
                   'weekly_price', 'monthly_price', 'is_business_travel_ready', 
                   'require_guest_profile_picture', 'require_guest_phone_verification']:
    features[tf_feature] = listings[tf_feature].map(lambda s: 0 if s == "f" else 1)

In [43]:
features.to_csv('../Data/listings_cleanTotal.csv', index=False)